In [1]:
import torch 
import torchvision
import torch.nn as nn 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 7.9MB/s 
     |████████████████████████████████| 3.0MB 26.7MB/s 
     |████████████████████████████████| 1.2MB 50.8MB/s 
     |████████████████████████████████| 901kB 50.3MB/s 


In [4]:
from transformers import pipeline

In [5]:
# Load data
#df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Donations.csv', parse_dates=['Donation Received Date'])


# Can't figure out how to read in the index as first column with pickle, so resetting 
# the index to first column then dropping it
#df.index = df.iloc[:,0].rename("")
#df = df.drop(columns=["Unnamed: 0"])
barley=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/barley_no_zero_duplicates.csv')
#bean=pd.read_csv('bean_no_zero_duplicates.csv')
corn_maize=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/corn_maize_no_zero_duplicates.csv')
#edamame=pd.read_csv('edamame_no_zero_duplicates.csv')
#lentil=pd.read_csv('lentil_no_zero_duplicates.csv')
millet=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/millet_no_zero_duplicates.csv')
#nut=pd.read_csv('nut_no_zero_duplicates.csv')
oat=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/oat_no_zero_duplicates.csv')
#pea=pd.read_csv('pea_no_zero_duplicates.csv')
potato=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/potato_no_zero_duplicates.csv')
quinoa=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/quinoa_no_zero_duplicates.csv')
rice=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/rice_no_zero_duplicates.csv')
rye=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/rye_no_zero_duplicates.csv')
sorghum=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/sorghum_no_zero_duplicates.csv')
soybean=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/soybean_no_zero_duplicates.csv')
wheat=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/wheat_no_zero_duplicates.csv')
legume_nuts_no_peanuts=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/legume_nuts_no_peanuts_no_zero_duplicates.csv')
legume_peanuts=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/legume_peanuts_no_zero_duplicates.csv')
legume_other=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/legume_other_no_zero_duplicates.csv')
whole=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Investment_gap/whole_removing duplicated project ids.csv')

#print('Data loaded')


In [6]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification',
                      model='facebook/bart-large-mnli', device=0)


Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
#candidate_labels = ['production', 'biological stress', 'environment','cultivation and management', 'human health', 'animal', 'bio-fuel', 'non-food']

In [8]:
candidate_labels = ['crop yield', 'crop growth', 'breeding', 'pest', 'crop disease', 'resistance', 'stress', 'virus', 'pathogen',
                    'environment', 'greenhouse gas', 'climate change', 'ecosystem', 'nitrogen', 
                    'phosphorous', 'fertilizer', 'water', 'cover crop', 'cropping system', 'crop rotation', 'soil', 
                    'tillage', 'human health', 'human diet', 'human nutrition', 'livestock', 
                    'cattle', 'hog', 'pig', 'chicken', 'fish', 'animal feed', 'meat', 'dairy', 'beef', 'forage', 'animal', 'biofuel', 'bioenergy']
col_names = ['Abstract', 'Project_ID', 'Production: crop yield', 'Production: crop growth', 'Production: breeding', 
             'Bio_stress: pest', 'Bio_stress: crop disease', 'Bio_stress: resistance', 'Bio_stress: stress', 'Bio_stress: virus', 'Bio_stress: pathogen',
             'Environment: environment', 'Environment: greenhouse gas', 'Environment: climate change', 'Environment: ecosystem', 
             'Cultivation: nitrogen', 'Cultivation: phosphorous', 'Cultivation: fertilizer', 'Cultivation: water', 'Cultivation: cover crop', 
             'Cultivation: cropping system', 'Cultivation: crop rotation', 'Cultivation: soil', 'Cultivation: tillage', 
             'Human_health: human health', 'Human_health: human diet', 'Human_health: human nutrition', 
             'Animal: livestock', 'Animal: cattle', 'Animal: hog', 'Animal: pig', 'Animal: chicken', 'Animal: fish', 'Animal: animal feed', 
             'Animal: meat', 'Animal: dairy', 'Animal: beef', 'Animal: forage', 'Animal: animal', 
             'Bioenegry: biofuel', 'Bioenegry: bioenergy']

In [ ]:
data=[whole]
category=['TM_all_abstracts']
dhara=0
for df in data:
# Drop rows that are missing in "body" column
  #df= df.sample(frac=0.03, replace=False, random_state=1)
  df.reset_index(drop=True, inplace=True)
  print(category[dhara])
  df['TEXT'] = df['ABSTRACT'].astype(str) + df[
      'PROJECT_TERMS'].astype(str) + df['PROJECT_TITLE'].astype(str)
  df['TEXT'] = df['TEXT'].astype(str)
  df.fillna(' ', inplace=True)  
  df['TEXT_PROCESSED'] = df['TEXT'].map(
      lambda x: re.sub('[,\.!?]', '', x))
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].map(
      lambda x: x.lower())
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].replace('\d+','', regex=True)
  d={'Abstract': [],'Project_ID': [], 'crop yield' : [], 'crop growth' : [], 'breeding' : [], 'pest' : [], 'crop disease' : [], 'resistance' : [],
     'stress' : [], 'virus' : [], 'pathogen' : [], 'environment' : [], 'greenhouse gas' : [], 'climate change' : [], 'ecosystem' : [], 
     'nitrogen' : [], 'phosphorous' : [], 'fertilizer' : [], 'water' : [], 'cover crop' : [], 'cropping system' : [], 'crop rotation' : [], 'soil' : [],
     'tillage' : [], 'human health' : [], 'human diet' : [], 'human nutrition' : [], 'livestock' : [], 
     'cattle' : [], 'hog' : [], 'pig' : [], 'chicken' : [], 'fish' : [], 'animal feed' : [], 'meat' : [], 'dairy' : [], 'beef' : [], 
     'forage' : [], 'animal' : [], 'biofuel' : [], 'bioenergy' : []} 
  print(df.shape)
  for i in range(len(df)):
    print(i)
    c=classifier(df['TEXT_PROCESSED'][i], candidate_labels)
    d['Abstract'].append(c['sequence'])
    d['Project_ID'].append(df['PROJECT_ID'][i])
    for j in candidate_labels:
      t=c['labels'].index(j)
      d[j].append(c['scores'][t])
  
  d=pd.DataFrame.from_dict(d)
  d.columns = col_names
  filename=category[dhara]+'.csv'
  d.to_csv(filename,index=False)
  !cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/Topic_Modelling"
  dhara+=1



  #c[0]['labels']

In [ ]:
data=[legume_nuts_no_peanuts,legume_peanuts,legume_other]
category=['TM_Legume_nuts_no_peanuts','TM_Legume_peanuts','TM_Other']
dhara=0
for df in data:
# Drop rows that are missing in "body" column
  print(category[dhara])
  df['TEXT'] = df['ABSTRACT'].astype(str) + df[
      'PROJECT_TERMS'].astype(str) + df['PROJECT_TITLE'].astype(str)
  df['TEXT'] = df['TEXT'].astype(str)
  df.fillna(' ', inplace=True)  
  df['TEXT_PROCESSED'] = df['TEXT'].map(
      lambda x: re.sub('[,\.!?]', '', x))
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].map(
      lambda x: x.lower())
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].replace('\d+','', regex=True)
  d={'Abstract': [],'Project_ID': [],  'crop yield' : [], 'crop growth' : [], 'breeding' : [], 'pest' : [], 'crop disease' : [], 'resistance' : [],
     'stress' : [], 'virus' : [], 'pathogen' : [], 'environment' : [], 'greenhouse gas' : [], 'climate change' : [], 'ecosystem' : [], 
     'nitrogen' : [], 'phosphorous' : [], 'fertilizer' : [], 'water' : [], 'cover crop' : [], 'cropping system' : [], 'crop rotation' : [], 'soil' : [],
     'tillage' : [], 'human health' : [], 'human diet' : [], 'human nutrition' : [], 'livestock' : [], 
     'cattle' : [], 'hog' : [], 'pig' : [], 'chicken' : [], 'fish' : [], 'animal feed' : [], 'meat' : [], 'dairy' : [], 'beef' : [], 
     'forage' : [], 'animal' : [], 'biofuel' : [], 'bioenergy' : []} 
  print(df.shape)
  for i in range(len(df)):
    print(i)
    c=classifier(df['TEXT_PROCESSED'][i], candidate_labels)
    d['Abstract'].append(c['sequence'])
    d['Project_ID'].append(df['PROJECT_ID'][i])
    for j in candidate_labels:
      t=c['labels'].index(j)
      d[j].append(c['scores'][t])
  d=pd.DataFrame.from_dict(d)
  d.columns = col_names
  filename=category[dhara]+'.csv'
  d.to_csv(filename, index=False)
  !cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/Topic_Modelling"
  dhara+=1



  #c[0]['labels']

TM_Legume_nuts_no_peanuts
(1570, 28)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
2

In [ ]:
from google.colab import files
filename='whole_zero_shot_results.csv'
d.to_csv(filename)
!cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap"
files.download("whole_zero_shot_results.csv")

In [ ]:
data=[barley,corn_maize,millet,oat,legume,potato,quinoa,rice,rye]
category=['TM_Barley','TM_Corn_maize','TM_Millet','TM_Oat','TM_Legume','TM_Potato','TM_Quinoa','TM_Rice','TM_Rye']
dhara=0
for df in data:
# Drop rows that are missing in "body" column
  print(category[dhara])
  df['TEXT'] = df['ABSTRACT'].astype(str) + df[
      'PROJECT_TERMS'].astype(str) + df['PROJECT_TITLE'].astype(str)
  df['TEXT'] = df['TEXT'].astype(str)
  df.fillna(' ', inplace=True)  
  df['TEXT_PROCESSED'] = df['TEXT'].map(
      lambda x: re.sub('[,\.!?]', '', x))
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].map(
      lambda x: x.lower())
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].replace('\d+','', regex=True)
  d={'Abstract': [],'Project_ID': [],  'crop yield' : [], 'crop growth' : [], 'breeding' : [], 'pest' : [], 'crop disease' : [], 'resistance' : [],
     'stress' : [], 'virus' : [], 'pathogen' : [], 'environment' : [], 'greenhouse gas' : [], 'climate change' : [], 'ecosystem' : [], 
     'nitrogen' : [], 'phosphorous' : [], 'fertilizer' : [], 'water' : [], 'cover crop' : [], 'cropping system' : [], 'crop rotation' : [], 'soil' : [],
     'tillage' : [], 'human health' : [], 'human diet' : [], 'human nutrition' : [], 'livestock' : [], 
     'cattle' : [], 'hog' : [], 'pig' : [], 'chicken' : [], 'fish' : [], 'animal feed' : [], 'meat' : [], 'dairy' : [], 'beef' : [], 
     'forage' : [], 'animal' : [], 'biofuel' : [], 'bioenergy' : []} 
  print(df.shape)
  for i in range(len(df)):
    print(i)
    c=classifier(df['TEXT_PROCESSED'][i], candidate_labels)
    d['Abstract'].append(c['sequence'])
    d['Project_ID'].append(df['PROJECT_ID'][i])
    for j in candidate_labels:
      t=c['labels'].index(j)
      d[j].append(c['scores'][t])
  d=pd.DataFrame.from_dict(d)
  d.columns = col_names
  filename=category[dhara]+'.csv'
  d.to_csv(filename, index=False)
  !cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/Topic_Modelling"
  dhara+=1



  #c[0]['labels']

TM_Barley
(186, 28)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
TM_Corn_maize
(834, 28)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [ ]:
data=[sorghum,soybean,wheat]
category=['TM_Sorghum','TM_Soybean','TM_Wheat']
dhara=0
for df in data:
# Drop rows that are missing in "body" column
  print(category[dhara])
  df['TEXT'] = df['ABSTRACT'].astype(str) + df[
      'PROJECT_TERMS'].astype(str) + df['PROJECT_TITLE'].astype(str)
  df['TEXT'] = df['TEXT'].astype(str)
  df.fillna(' ', inplace=True)  
  df['TEXT_PROCESSED'] = df['TEXT'].map(
      lambda x: re.sub('[,\.!?]', '', x))
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].map(
      lambda x: x.lower())
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].replace('\d+','', regex=True)
  d={'Abstract': [],'Project_ID': [],  'crop yield' : [], 'crop growth' : [], 'breeding' : [], 'pest' : [], 'crop disease' : [], 'resistance' : [],
     'stress' : [], 'virus' : [], 'pathogen' : [], 'environment' : [], 'greenhouse gas' : [], 'climate change' : [], 'ecosystem' : [], 
     'nitrogen' : [], 'phosphorous' : [], 'fertilizer' : [], 'water' : [], 'cover crop' : [], 'cropping system' : [], 'crop rotation' : [], 'soil' : [],
     'tillage' : [], 'human health' : [], 'human diet' : [], 'human nutrition' : [], 'livestock' : [], 
     'cattle' : [], 'hog' : [], 'pig' : [], 'chicken' : [], 'fish' : [], 'animal feed' : [], 'meat' : [], 'dairy' : [], 'beef' : [], 
     'forage' : [], 'animal' : [], 'biofuel' : [], 'bioenergy' : []}  
  print(df.shape)
  for i in range(len(df)):
    print(i)
    c=classifier(df['TEXT_PROCESSED'][i], candidate_labels)
    d['Abstract'].append(c['sequence'])
    d['Project_ID'].append(df['PROJECT_ID'][i])
    for j in candidate_labels:
      t=c['labels'].index(j)
      d[j].append(c['scores'][t])
  d=pd.DataFrame.from_dict(d)
  d.columns = col_names
  filename=category[dhara]+'.csv'
  d.to_csv(filename, index=False)
  !cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/Topic_Modelling"
  dhara+=1



  #c[0]['labels']

TM_Sorghum
(208, 28)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
TM_Soybean
(569, 28)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [ ]:
data=[whole]
category=['TM_all_abstracts']
dhara=0
for df in data:
# Drop rows that are missing in "body" column
  print(category[dhara])
  df['TEXT'] = df['ABSTRACT'].astype(str) + df[
      'PROJECT_TERMS'].astype(str) + df['PROJECT_TITLE'].astype(str)
  df['TEXT'] = df['TEXT'].astype(str)
  df.fillna(' ', inplace=True)  
  df['TEXT_PROCESSED'] = df['TEXT'].map(
      lambda x: re.sub('[,\.!?]', '', x))
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].map(
      lambda x: x.lower())
  df['TEXT_PROCESSED'] = df['TEXT_PROCESSED'].replace('\d+','', regex=True)
  d={'Abstract': [],'Project_ID': [], 'crop yield' : [], 'crop growth' : [], 'breeding' : [], 'pest' : [], 'crop disease' : [], 'resistance' : [],
     'stress' : [], 'virus' : [], 'pathogen' : [], 'environment' : [], 'greenhouse gas' : [], 'climate change' : [], 'ecosystem' : [], 
     'nitrogen' : [], 'phosphorous' : [], 'fertilizer' : [], 'water' : [], 'cover crop' : [], 'cropping system' : [], 'crop rotation' : [], 'soil' : [],
     'tillage' : [], 'human health' : [], 'human diet' : [], 'human nutrition' : [], 'livestock' : [], 
     'cattle' : [], 'hog' : [], 'pig' : [], 'chicken' : [], 'fish' : [], 'animal feed' : [], 'meat' : [], 'dairy' : [], 'beef' : [], 
     'forage' : [], 'animal' : [], 'biofuel' : [], 'bioenergy' : []}  
  print(df.shape)
  for i in range(len(df)):
    print(i)
    c=classifier(df['TEXT_PROCESSED'][i], candidate_labels)
    d['Abstract'].append(c['sequence'])
    d['Project_ID'].append(df['PROJECT_ID'][i])
    for j in candidate_labels:
      t=c['labels'].index(j)
      d[j].append(c['scores'][t])
  d=pd.DataFrame.from_dict(d)
  d.columns = col_names
  filename=category[dhara]+'.csv'
  d.to_csv(filename, index=False)
  !cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/Topic_Modelling"
  dhara+=1



  #c[0]['labels']

In [ ]:
filename='millet_zero_shot_results.csv'
d.to_csv(filename)
!cp $filename "/content/drive/My Drive/Colab Notebooks/Investment_gap/results"
from google.colab import files

files.download("millet_zero_shot_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>